In [12]:
import pandas as pd
import numpy as np
import tensorflow as tf

from sklearn.preprocessing import LabelEncoder
from sklearn.cross_validation import StratifiedShuffleSplit
from sklearn.preprocessing import StandardScaler

In [13]:
# 读取数据
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# Swiss army knife function to organize the data

# 提取label，并编码成数字
def encode(train, test):
    le = LabelEncoder().fit(train.species) 
    labels = le.transform(train.species)           # encode species strings
    classes = list(le.classes_)                    # save column names for submission
    test_ids = test.id                             # save test ids for submission
    
    train = train.drop(['species', 'id'], axis=1)  
    test = test.drop(['id'], axis=1)
    
    scaler = StandardScaler().fit(train.values)
    train = scaler.transform(train.values)
    test = scaler.transform(test.values)
    
    return train, labels, test, test_ids, classes

train, labels, test, test_ids, classes = encode(train, test)

In [14]:
# 在train数据里划分训练测试数据
sss = StratifiedShuffleSplit(labels, 10, test_size=0.2, random_state=23)

for train_index, test_index in sss:
    X_train, X_test = train[train_index], train[test_index]
    y_train, y_test = labels[train_index], labels[test_index]

In [15]:
def label_trans(x):
    output = np.zeros(99)
    output[x] = 1
    return output

In [19]:
y_train_soft = map(label_trans, y_train)
y_train_soft = np.array(y_train_soft)
y_test_soft = map(label_trans, y_test)
y_test_soft = np.array(y_test_soft)

In [20]:
# configuration variables
input_vec_size = lstm_size = 64
time_step_size = 3
train_size = batch_size = 792

def init_weights(shape):
    return tf.Variable(tf.random_normal(shape, stddev=0.01))

In [21]:
def model(X, W, B, lstm_size):
    # X, input shape: (batch_size, time_step_size, input_vec_size)
    XT = tf.transpose(X, [1, 0, 2])  # permute time_step_size and batch_size
    # XT shape: (time_step_size, batch_size, input_vec_size)
    XR = tf.reshape(XT, [-1, lstm_size]) # each row has input for each lstm cell (lstm_size=input_vec_size)
    # XR shape: (time_step_size * batch_size, input_vec_size)
    X_split = tf.split(0, time_step_size, XR) # split them to time_step_size (28 arrays)
    # Each array shape: (batch_size, input_vec_size)

    # Make lstm with lstm_size (each input vector size)
    lstm = tf.nn.rnn_cell.BasicLSTMCell(lstm_size, forget_bias=1.0, state_is_tuple=True)

    # Get lstm cell output, time_step_size (28) arrays with lstm_size output: (batch_size, lstm_size)
    outputs, _states = tf.nn.rnn(lstm, X_split, dtype=tf.float32)

    # Linear activation
    # Get the last output
    return tf.matmul(outputs[-1], W) + B, lstm.state_size # State size to initialize the stat

In [22]:
X = tf.placeholder("float", [None, 3, 64])
Y = tf.placeholder("float", [None, 99])

# get lstm_size and output 10 labels
W = init_weights([lstm_size, 99])
B = init_weights([99])

py_x, state_size = model(X, W, B, lstm_size)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(py_x, Y))
train_op = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cost)
yprob_op = tf.nn.softmax(py_x)
predict_op = tf.argmax(py_x, 1)

In [24]:
X_train = X_train.reshape(-1,3,64)

In [25]:
X_test = X_test.reshape(-1,3,64)

In [36]:
train = train.reshape(-1,3,64)
test = test.reshape(-1,3,64)

In [41]:
# Launch the graph in a session
with tf.Session() as sess:
    # you need to initialize all variables
    tf.initialize_all_variables().run()

    for i in range(1000):
        sess.run(train_op,feed_dict={X: X_train, Y: y_train_soft})
        
        #print(i, np.mean(y_test ==
                         #sess.run(predict_op, feed_dict={X: X_test})))
        
    label_soft = map(label_trans, labels)
    label_soft = np.array(label_soft)
    sess.run(train_op, feed_dict={X:train, Y: label_soft})
    yprob = sess.run(yprob_op, feed_dict={X:test})
    print(yprob)
    submission = pd.DataFrame(yprob, index=test_ids, columns=classes)
    submission = submission.reset_index()
    submission.to_csv("rnn_submission.csv", index=False)

[[  4.54273987e-08   1.73387363e-07   9.99206051e-10 ...,   7.59505525e-09
    5.60039507e-06   4.52898412e-08]
 [  1.40732475e-08   5.39953904e-09   3.31432517e-08 ...,   2.48090128e-05
    1.17251338e-08   2.22896013e-08]
 [  2.44738743e-07   9.99342501e-01   1.94365430e-08 ...,   1.31570861e-13
    9.93593403e-08   3.20608997e-05]
 ..., 
 [  5.96643304e-07   3.28673444e-08   1.66487037e-08 ...,   1.09860288e-07
    1.73401602e-07   9.04308521e-08]
 [  2.82783237e-08   4.87330265e-09   2.72585257e-05 ...,   1.20873813e-06
    6.75728318e-09   2.92259784e-07]
 [  1.38224371e-11   7.99293076e-09   3.18882144e-07 ...,   1.21144810e-08
    1.69656639e-12   6.82837609e-09]]
